<a href="https://colab.research.google.com/github/haidoro/Titanic/blob/master/Titanic_svc3_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titanic分類

In [0]:
# データ加工・処理・分析ライブラリ
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series,DataFrame
import pandas as pd

# 可視化ライブラリ
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline
sns.set()

# 機械学習ライブラリ
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# 小数第3位まで表示
%precision 3

pd.set_option('display.max_rows', 900)

In [3]:
# インポート
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Titanicデータを読み込み
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print('Train欠損値',train.isnull().sum(),len(train))
print('Test欠損値',test.isnull().sum(),len(train))


Train欠損値 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64 891
Test欠損値 PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64 891


In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [0]:
train['Age'] = train['Age'].fillna(train['Fare'].map(lambda x: 50 if x > 70 else 20))
train = train.dropna(subset=['Embarked'])
age_bins =[0,10,20,30,40,50,90]
train['age_group'] = pd.cut(train['Age'],age_bins)

In [7]:
train_X = pd.get_dummies(train[['Survived','Embarked','Pclass', 'SibSp', 'Parch','age_group']],dummy_na=True)
train_X['sex'] = train['Sex'].apply(lambda x : 0 if x=='male' else 1 )
train_X.head()

,Survived,Pclass,SibSp,Parch,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,"age_group_(0, 10]","age_group_(10, 20]","age_group_(20, 30]","age_group_(30, 40]","age_group_(40, 50]","age_group_(50, 90]",age_group_nan,sex
0,0,3,1,0,0,0,1,0,0,0,1,0,0,0,0,0
1,1,1,1,0,1,0,0,0,0,0,0,1,0,0,0,1
2,1,3,0,0,0,0,1,0,0,0,1,0,0,0,0,1
3,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,1
4,0,3,0,0,0,0,1,0,0,0,0,1,0,0,0,0


In [0]:
train_y = train_X['Survived']
train_X = train_X.drop('Survived',axis=1)

# 訓練データとテストデータに分ける
X_train, X_test, y_train, y_test = train_test_split(
    train_X, train_y, random_state=0)

In [9]:
# SVCモデルの設定
model = SVC(C=1.9,gamma=0.1)

model.fit(X_train, y_train)
print('train score', model.score(X_train, y_train))
print('test score', model.score(X_test, y_test))


train score 0.8438438438438438
test score 0.7892376681614349


In [0]:
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())
test['Age'] = test['Age'].fillna(test['Fare'].map(lambda x: 50 if x > 70 else 20))
test['age_group'] = pd.cut(test['Age'],age_bins)

In [11]:
test_X = pd.get_dummies(test[['Embarked','Pclass', 'SibSp', 'Parch','age_group']],dummy_na=True)
test_X['sex'] = test['Sex'].apply(lambda x : 0 if x=='male' else 1 )
test_X.head()

,Pclass,SibSp,Parch,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,"age_group_(0, 10]","age_group_(10, 20]","age_group_(20, 30]","age_group_(30, 40]","age_group_(40, 50]","age_group_(50, 90]",age_group_nan,sex
0,3,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1,3,1,0,0,0,1,0,0,0,0,0,1,0,0,1
2,2,0,0,0,1,0,0,0,0,0,0,0,1,0,0
3,3,0,0,0,0,1,0,0,0,1,0,0,0,0,0
4,3,1,1,0,0,1,0,0,0,1,0,0,0,0,1


In [0]:
prediction = model.predict(test_X)

In [13]:
test['Survived'] = prediction
my_submission = test[['PassengerId', 'Survived']]
my_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [0]:
my_submission.to_csv('./submission.csv',index=False)